In [57]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [58]:
def crop_image(img, start_x, start_y, end_x, end_y):
    # This is useful to focus on a specific area of the image by removing irrelevant parts, which can help in reducing the size of the image and focusing on key features.
    return img[start_y:end_y, start_x:end_x]

def flip_image(img, flip_type):
    # Flipping is a common operation in data augmentation to create variations of the image, ensuring the model becomes more robust to different orientations of objects.
    return cv2.flip(img, flip_type)

def rotate_image(img, angle, scale=1.0):
    # This transformation simulates the effect of objects being captured from different angles which helps improve the robustness of a model by introducing rotational variance to the training data.
    rows, cols = img.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
    return cv2.warpAffine(img, rotation_matrix, (cols, rows))

def resize_image(img, width, height):
    # By doing this, standardize all images to a consistent size, regardless of their original dimensions.
    dim = (width, height)
    return cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

def translate_image(img, shift_x, shift_y):
    # This transformation simulates small movements of the camera or object in the scene, making the model more resilient to such shifts during real-world use.
    translation_matrix = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    return cv2.warpAffine(img, translation_matrix, (img.shape[1], img.shape[0]))

def shear_image(img, shear_x, shear_y):
    # Shearing distorts the shape of the image in a controlled manner. This simulates the effect of objects being captured from an angle or a perspective view and increases model robustness to such distortions.
    rows, cols = img.shape[:2]
    shear_matrix = np.float32([[1, shear_x, 0], [shear_y, 1, 0]])
    return cv2.warpAffine(img, shear_matrix, (cols, rows))

def process_and_save_images(source_dir, destination_dir, resize_dimen):
    for filename in os.listdir(source_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(source_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            cropped = crop_image(img, 100, 100, 277, 243)
            flipped = flip_image(img, 0)
            rotated = rotate_image(img, -90, 0.5)
            resized = resize_image(img, resize_dimen[0], resize_dimen[1])
            translated = translate_image(img, 50, 30)
            sheared = shear_image(img, 0.2, 0.0)

            base_name, ext = os.path.splitext(filename)

            cropped_filename = f"cropped_{base_name}{ext}"
            cropped_destination_path = os.path.join(destination_dir, cropped_filename)
            plt.imsave(cropped_destination_path, cropped)

            flipped_filename = f"flipped_{base_name}{ext}"
            flipped_destination_path = os.path.join(destination_dir, flipped_filename)
            plt.imsave(flipped_destination_path, flipped)

            rotated_filename = f"rotated_{base_name}{ext}"
            rotated_destination_path = os.path.join(destination_dir, rotated_filename)
            plt.imsave(rotated_destination_path, rotated)

            resized_filename = f"resized_{base_name}{ext}"
            resized_destination_path = os.path.join(destination_dir, resized_filename)
            plt.imsave(resized_destination_path, resized)

            translated_filename = f"translated_{base_name}{ext}"
            translated_destination_path = os.path.join(destination_dir, translated_filename)
            plt.imsave(translated_destination_path, translated)

            sheared_filename = f"sheared_{base_name}{ext}"
            sheared_destination_path = os.path.join(destination_dir, sheared_filename)
            plt.imsave(sheared_destination_path, sheared)

    print("Image transformations, saving completed.")

In [59]:
source_dir = "/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/SourceFiles"
destination_dir = "/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/DestFiles"
resize_dimen = (1024, 1024)
os.makedirs(destination_dir, exist_ok=True)
process_and_save_images(source_dir, destination_dir, resize_dimen)

Image transformations, saving, and plotting completed.


In [60]:
def salt_and_pepper_noise(image, prob):
    noisy_image = np.copy(image)
    num_salt = np.ceil(prob * image.size * 0.5).astype(int)  # Ensure this is an integer
    num_pepper = np.ceil(prob * image.size * 0.5).astype(int)  # Ensure this is an integer

    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape]
    noisy_image[salt_coords[0], salt_coords[1]] = 255

    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape]
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0
    return noisy_image

def gaussian_noise(image, mean=0, std=50):
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return noisy_image

def apply_noise(image, prob_salt_pepper=0.05, gaussian_std=50):
    salt_pepper = salt_and_pepper_noise(image, prob_salt_pepper)
    gaussian = gaussian_noise(image, std=gaussian_std)

    return salt_pepper, gaussian

def plot_histograms_side_by_side(salt_pepper, gaussian, filename):
    plt.figure(figsize=(20, 6))

    plt.subplot(1, 2, 1)
    plt.hist(salt_pepper.flatten(), bins=50, density=True, alpha=0.7, color='r')
    plt.title(f'Salt-and-Pepper Histogram - {filename}')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(gaussian.flatten(), bins=50, density=True, alpha=0.7, color='g')
    plt.title(f'Gaussian Noise Histogram - {filename}')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

def plot_noisy_images_with_histograms(original, salt_pepper, gaussian, filename):
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))

    axes[0].set_title(f'Original - {filename}')
    axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')

    axes[1].set_title(f'Salt-and-Pepper Noise - {filename}')
    axes[1].imshow(cv2.cvtColor(salt_pepper, cv2.COLOR_BGR2RGB))
    axes[1].axis('off')

    axes[2].set_title(f'Gaussian Noise - {filename}')
    axes[2].imshow(cv2.cvtColor(gaussian, cv2.COLOR_BGR2RGB))
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    plot_histograms_side_by_side(salt_pepper, gaussian, filename)

def process_images(source_dir, destination_dir, resize_dimen):
    for filename in os.listdir(source_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(source_dir, filename)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            img_resized = cv2.resize(img, resize_dimen)

            salt_pepper, gaussian = apply_noise(img_resized)
            base_name, ext = os.path.splitext(filename)

            salt_pepper_filename = f"salt_pepper_{base_name}{ext}"
            salt_pepper_path = os.path.join(destination_dir, salt_pepper_filename)
            plt.imsave(salt_pepper_path, cv2.cvtColor(salt_pepper, cv2.COLOR_BGR2RGB))

            gaussian_filename = f"gaussian_{base_name}{ext}"
            gaussian_path = os.path.join(destination_dir, gaussian_filename)
            plt.imsave(gaussian_path, cv2.cvtColor(gaussian, cv2.COLOR_BGR2RGB))

            plot_noisy_images_with_histograms(img_resized, salt_pepper, gaussian, filename)

    print("Image noise transformation, saving, plotting, and histogram display completed.")

source_dir = "/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/SourceFiles"
destination_dir = "/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/DestFiles"
resize_dimen = (1024, 1024)

os.makedirs(destination_dir, exist_ok=True)
process_images(source_dir, destination_dir, resize_dimen)


Output hidden; open in https://colab.research.google.com to view.

In [61]:
def apply_transformations(img, resize_dimen):
    bgr = img
    resized_img = cv2.resize(bgr, resize_dimen)
    rgb = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    grayscale = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    inverted = cv2.bitwise_not(resized_img)
    hsv = cv2.cvtColor(resized_img, cv2.COLOR_BGR2HSV)

    return resized_img, rgb, grayscale, inverted, hsv
def plot_transformed_images(img, rgb, grayscale, inverted, hsv, filename):
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(20, 10))

    ax1.set_title(f'BGR - {filename}')
    ax1.imshow(img)
    ax1.axis('off')

    ax2.set_title(f'RGB - {filename}')
    ax2.imshow(rgb)
    ax2.axis('off')

    ax3.set_title(f'Grayscale - {filename}')
    ax3.imshow(grayscale, cmap='gray')
    ax3.axis('off')

    ax4.set_title(f'Inverted - {filename}')
    ax4.imshow(inverted)
    ax4.axis('off')

    ax5.set_title(f'HSV - {filename}')
    ax5.imshow(hsv)
    ax5.axis('off')

    plt.tight_layout()
    plt.show()
def process_images(source_dir, destination_dir, resize_dimen):
    for filename in os.listdir(source_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(source_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            bgr, rgb, grayscale, inverted, hsv = apply_transformations(img, resize_dimen)

            base_name, ext = os.path.splitext(filename)

            bgr_filename = f"bgr_{base_name}{ext}"
            bgr_destination_path = os.path.join(destination_dir, bgr_filename)
            plt.imsave(bgr_destination_path, bgr)

            rgb_filename = f"rgb_{base_name}{ext}"
            rgb_destination_path = os.path.join(destination_dir, rgb_filename)
            plt.imsave(rgb_destination_path, rgb)

            grayscale_filename = f"grayscale_{base_name}{ext}"
            grayscale_destination_path = os.path.join(destination_dir, grayscale_filename)
            plt.imsave(grayscale_destination_path, grayscale, cmap='gray')

            inverted_filename = f"inverted_{base_name}{ext}"
            inverted_destination_path = os.path.join(destination_dir, inverted_filename)
            plt.imsave(inverted_destination_path, inverted)

            hsv_filename = f"hsv_{base_name}{ext}"
            hsv_destination_path = os.path.join(destination_dir, hsv_filename)
            plt.imsave(hsv_destination_path, hsv)

            plot_transformed_images(bgr, rgb, grayscale, inverted, hsv, filename)

    print("Image transformations, saving, and plotting completed.")

In [62]:
source_dir = "/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/SourceFiles"
destination_dir="/content/drive/MyDrive/CSE463-ComputerVision/24241266_PravakarDas_Lab1/Task2/DestFiles"
resize_dimen = (1024, 1024)

os.makedirs(destination_dir, exist_ok=True)
process_images(source_dir, destination_dir, resize_dimen)


Output hidden; open in https://colab.research.google.com to view.